In [1]:
# libs import
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import *
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd
import re

from time import sleep
from datetime import datetime
import os


In [2]:
# configurações chromedriver
chrome_options = Options()
chrome_options.add_experimental_option(
    'excludeSwitches', ['enable-logging'])
chrome_options.add_argument('--lang=pt-BR')
chrome_options.add_argument('--disable-notifications')

In [3]:
# parâmetros busca
destino = "Balneário Camboriú"
checkin = "2023-12-01" #AAAA-MM-DD
checkout = "2023-12-05"
rooms = "1"
ad = "2"
chd = "0"

In [4]:
# lista organização dados extraídos
organized_data = []

# scraper
class Scraper:

    def run(self):
        self.coleta_dados()
    
    def coleta_dados(self):              

        # inicializa driver navegador
        self.driver = webdriver.Chrome(options=chrome_options)
        self.driver.set_window_size(800, 700)        
        # url filtrada
        #self.link = f'''https://www.booking.com/searchresults.pt-br.html?label=gen173nr-1BCAEoggI46AdIM1gEaCCIAQGYAS24ARfIAQzYAQHoAQGIAgGoAgO4Ao_bsakGwAIB0gIkMWE2NmNhOGItNDUxMS00ODBlLWIyMzAtMmYyYTlhYTIwZWRj2AIF4AIB&sid=427e781d6854953190915ea83438a279&aid=304142&ss={destino}&lang=pt-br&src=index&dest_id=-627380&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=xb&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=b3b7a107f6dc01ec&ac_meta=GhBiM2I3YTEwN2Y2ZGMwMWVjIAAoATICeGI6BWJhbG5lQABKAFAA&checkin={checkin}&checkout={checkout}&group_adults={ad}&no_rooms={rooms}&group_children={chd}&sb_travel_purpose=leisure&nflt=ht_id%3D204'''      
        self.link = 'https://www.booking.com/searchresults.pt-br.html?label=gen173nr-1BCAEoggI46AdIM1gEaCCIAQGYAS24ARfIAQzYAQHoAQGIAgGoAgO4Ao_bsakGwAIB0gIkMWE2NmNhOGItNDUxMS00ODBlLWIyMzAtMmYyYTlhYTIwZWRj2AIF4AIB&sid=427e781d6854953190915ea83438a279&aid=304142&page=4&ss=Balne%C3%A1rio+Cambori%C3%BA&lang=pt-br&nflt=ht_id%3D204&offset=25'
        # abre página web
        self.driver.get(self.link)
        sleep(5)
        self.driver.find_element(By.XPATH, '/html/body').send_keys(Keys.ESCAPE)
        
        # extração estrutura HTML
        for p in range(1): # nº de páginas a percorrer     
            html_raw = self.driver.page_source
            
            # modelagem dados            
            # encontrar índice primeira ocorrência "pageName"
            match = re.search(r'"pageName"', html_raw)           
            # se string "pageName" for encontrada
            if match:
                start_index = match.start()
                # extrair parte relevante do texto
                relevant_text = html_raw[start_index:]
            else:
                print("String 'pageName' não encontrada.")

            # parte da string que contém informações sobre os hotéis
            html = relevant_text.replace('{"__typename":"StarRating","value":', '')
            
            # expressões regulares para extrair os campos específicos de cada hotel
            fields_to_extract = {
                #"pageName": r'"pageName":"(.*?)"',
                "totalScore": r'"totalScore":(\d+(?:\.\d+)?)',
                "reviewsCount": r'"reviewsCount":(\d+)',
                #"totalScoreTextTag": r'"totalScoreTextTag":{"__typename":"TranslationTag","translation":(null|".*?")',
                "StarRating": r'"starRating":(\d+|\d+\.\d+|null)',
                "displayName": r'"displayName":{"__typename":"TextWithTranslationTag","text":"(.*?)"',
                "geniusRateAvailable": r'"geniusRateAvailable":(true|false)',
                "highlighted": r'"highlighted":(true|false)',
                "preferred": r'"preferred":(true|false)',
                "preferredPlus": r'"preferredPlus":(true|false)',
                "isNewlyOpened": r'"isNewlyOpened":(true|false)',
                #"accommodationTypeId": r'"accommodationTypeId":(\d+)',
                #"id": r'"basicPropertyData":{"__typename":"BasicPropertyData","accommodationTypeId":\d+,"id":(\d+)',
                #"address": r'"address":"(.*?)"',
                #"city": r'"city":"(.*?)"',
                "displayLocation": r'"displayLocation":"(.*?)"'
            }
            
            # lista temp para armazenar dados de cada hotel
            hotel_data_list = []
            
            # iterar sobre padrões de expressões regulares para cada campo
            for field, pattern in fields_to_extract.items():
                data = re.findall(pattern, html)
                hotel_data_list.append((field, data))
            
            # organizar os dados de cada hotel
            num_hotels = min(len(data) for _, data in hotel_data_list)  # número de hotéis na lista
                        
            for i in range(num_hotels):
                hotel_info = {}
                for field, data in hotel_data_list:
                    hotel_info[field] = data[i] if i < len(data) else None
                organized_data.append(hotel_info)

            hotel_data_list.clear()
            
            # navegação páginas
            #botao_proximo = self.driver.find_element(By.XPATH,
            #    '//*[@id="search_results_table"]/div[2]/div/div/div[4]/div[2]/nav/div/div[3]/button')
            #botao_proximo.click()
            #print("Navegando para a próxima página")
            #sleep(5)
      
        self.driver.quit()


In [5]:
# execução scraper

scraper = Scraper()
scraper.run()


In [6]:
# dataframe
data_frame = pd.DataFrame(organized_data)


In [7]:
# reordenar as colunas
reord_columns = ["displayName", "displayLocation", "StarRating", "isNewlyOpened", "totalScore", 
                   "reviewsCount", "geniusRateAvailable", "preferred", "preferredPlus", "highlighted"]

# reindexar o dataframe
data_frame = data_frame.reindex(columns=reord_columns)


In [8]:
# exibir tabela
display(data_frame)


,displayName,displayLocation,StarRating,isNewlyOpened,totalScore,reviewsCount,geniusRateAvailable,preferred,preferredPlus,highlighted
0,BRUT by Slaviero Hotéis,Balneário Camboriú,4,false,8.4,1278,false,true,false,false
1,Hotel Suiça Faber,Balneário Camboriú,3,false,7.6,1610,false,true,false,false
2,Hotel Di Sica,Balneário Camboriú,null,false,8.3,907,false,true,false,false
3,Rezende Suítes Hotel,Balneário Camboriú,null,false,8.2,407,false,true,false,false
4,Hotel Gracher Praia,Balneário Camboriú,3,false,7.9,1370,false,true,false,false
5,Hotel Bhally,Balneário Camboriú,4,false,8.2,406,false,true,false,false
6,Pousada Ponta do Lobo,Balneário Camboriú,null,false,8.6,1008,false,true,false,false
7,Centromar Hotel,Balneário Camboriú,3,false,8.4,1014,false,false,false,false
8,Reserva Praia Hotel,Balneário Camboriú,4,false,9,857,false,true,false,false
9,Ibis Budget Balneario Camboriu,Balneário Camboriú,3,false,8.2,965,false,false,false,false


In [9]:
# download xlsx
now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

nome_arquivo = f'{destino}_{now}.xlsx'
data_frame.to_excel(nome_arquivo, index=False)


In [10]:
# WIP
#offset=0 = 1st page
#offset=25 = 2nd page
#offset=50 = 3rd page...